In [2]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Load and explore the dataset
df = pd.read_csv('../data/processed/updated_df.csv')
df.head()

,_id,AREA_NAME,HOOD_ID,ADMIN_DISTRICT,POPULATION_2023,ASSAULT_2014,ASSAULT_2015,ASSAULT_2016,ASSAULT_2017,ASSAULT_2018,...,THEFTOVER_RATE_2015,THEFTOVER_RATE_2016,THEFTOVER_RATE_2017,THEFTOVER_RATE_2018,THEFTOVER_RATE_2019,THEFTOVER_RATE_2020,THEFTOVER_RATE_2021,THEFTOVER_RATE_2022,THEFTOVER_RATE_2023,geometry
0,1,South Eglinton-Davisville,174,Old City of Toronto,21987,63,61,70,82,85,...,16.334532,21.590111,5.237247,15.232292,14.822866,24.197842,28.634151,14.006910,36.385136,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
1,2,North Toronto,173,Old City of Toronto,15077,45,52,43,52,55,...,25.911211,17.305529,16.500288,39.413528,30.234316,43.830814,14.241971,27.502750,46.428333,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
2,3,Dovercourt Village,172,Old City of Toronto,13837,56,57,79,94,94,...,22.218931,37.180248,22.146759,29.146021,21.698250,21.709240,14.541224,29.008631,21.681000,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
3,4,Junction-Wallace Emerson,171,Old City of Toronto,26240,154,157,166,157,157,...,20.975794,24.753496,24.404133,27.956388,15.754854,35.254025,31.309929,31.035419,34.298782,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."
4,5,Yonge-Bay Corridor,170,Old City of Toronto,14731,394,524,487,603,576,...,274.197968,233.301651,289.951111,349.539246,481.909271,259.798096,188.651871,358.826416,346.208679,"{'type': 'MultiPolygon', 'coordinates': [[[[-7..."


In [5]:
# Calculating the total crimes per neighborhood
df['Total_Crimes'] = df[['ASSAULT_2023', 'AUTOTHEFT_2023', 'BIKETHEFT_2023', 'BREAKENTER_2023',
       'HOMICIDE_2023', 'ROBBERY_2023', 'SHOOTING_2023', 'THEFTFROMMV_2023',
       'THEFTOVER_2023']].sum(axis=1)

# Create a mapping dictionary for custom titles
feature_titles = {
    'Total_Crimes': 'Total',
    'ASSAULT_2023': 'Assault',
    'AUTOTHEFT_2023': 'Auto theft',
    'BIKETHEFT_2023': 'Bike theft',
    'BREAKENTER_2023': 'Break and enter',
    'HOMICIDE_2023': 'Homicide',
    'ROBBERY_2023': 'Robbery',
    'SHOOTING_2023': 'Shooting',
    'THEFTFROMMV_2023': 'Theft from motor vehicles'
}

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the dashboard
app.layout = html.Div([
    html.H2("Toronto Crime Analysis Dashboard - Crime Analysis by District and Neighborhood",
            style={
            'backgroundColor': 'white',  # Set background color to white
            'padding': '10px',           # Add padding for better appearance
            'borderRadius': '2px',       # Round the corners slightly
            'textAlign': 'left'        # Align the text to the left
        }
    ),
    
    # Text above the crime metric dropdown
    html.Label("Select Metric:", style={'backgroundColor': 'white',
                                        'padding': '10px',
                                        'textAlign': 'left'
                                        }),

    # Dropdown to select the metric for analysis
    dcc.Dropdown(
        id='metric-dropdown',
        options=[
            {'label': 'Total Crimes', 'value': 'Total_Crimes'},
            {'label': 'Assaults', 'value': 'ASSAULT_2023'},
            {'label': 'Auto Theft', 'value': 'AUTOTHEFT_2023'},
            {'label': 'Bike Theft', 'value': 'BIKETHEFT_2023'},
            {'label': 'Break Enter', 'value': 'BREAKENTER_2023'},
            {'label': 'Homicide', 'value': 'HOMICIDE_2023'},
            {'label': 'Robbery', 'value': 'ROBBERY_2023'},
            {'label': 'Shooting', 'value': 'SHOOTING_2023'},
            {'label': 'Theft from Motor Vehicle', 'value': 'THEFTFROMMV_2023'}

        ],
        value='Total_Crimes'  # Default value
    ),
    
    # Text above the color scale dropdown
    html.Label("Select Color Palette:", style={'backgroundColor': 'white',
                                               'padding': '10px',
                                               'textAlign': 'left'
                                               }),
    
    # Dropdown to select the color scale
    dcc.Dropdown(
        id='color-scale-dropdown',
        options=[
            {'label': 'Yellow-Orange-Red', 'value': 'YlOrRd'},
            {'label': 'Matter', 'value': 'matter'},
            {'label': 'Blues', 'value': 'Blues'}
        ],
        value='YlOrRd'  # Default value
    ),
    
    # Placeholder for the treemap
    dcc.Graph(id='treemap-graph')
])

# Callback to update the treemap based on user input
@app.callback(
    Output('treemap-graph', 'figure'),
    Input('metric-dropdown', 'value'),
    Input('color-scale-dropdown', 'value')
)
def update_treemap(selected_metric, selected_color_scale):
    # Calculate the total number of crimes for the selected metric
    total_crimes = df[selected_metric].sum()

    # Format the total_crimes number with commas
    total_crimes_formatted = f"{total_crimes:,.0f}"

    # Create the treemap
    fig = px.treemap(df,
                     path=['ADMIN_DISTRICT', 'AREA_NAME',],
                     values=selected_metric,  # Dynamic based on dropdown selection
                     color=selected_metric,   # Dynamic based on dropdown selection
                     hover_data={selected_metric: True},
                     color_continuous_scale=selected_color_scale,
                     title=f"{feature_titles[selected_metric]} crimes committed in 2023: {total_crimes_formatted}")
    

    # Customize hover template to show only neighborhood name and crime count
    fig.update_traces(
        hovertemplate="<b>%{label}</b><br>Total crimes: %{value:,}<br><extra></extra>"
    )

    fig.update_layout(font_family="Times New Roman")
    
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)
